In [2]:
from itertools import count

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier


In [3]:
df = pd.read_csv("dataset.csv")
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


1. Si vuole predire il valore di range di prezzo sulla base degli attributi presenti nel dataset. Ricaricare il dataset originale, eliminare eventuali attributi inutili (giustificare la scelta), eliminare le eventuali istanze che contengono valori nulli, e dividere il dataset in modo che 3/4 degli elementi siano contenuti in un nuovo dataset “train” e 1/4 nel dataset “test” preservando le proporzioni delle classi nella colonna target.


In [4]:
y = df["price_range"]
X= df.drop("price_range", axis=1)
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

Allenare il train con il modello DecisionTree e valutare l’accuracy ottenuta calcolata sia sul dataset train sia sul dataset test. Confrontare i risultati ottenuti con quelli ottenuti con una predizione basata sul modello KNeighborsClassifier. Effettuare alcune considerazioni sui risultati ottenuti, tenendo in considerazione anche l’analisi della confusion matrix e la predizione effettuata da un dummy classifier a scelta. (punti 4)

In [5]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
y_testpred = model.predict(X_test)
print(accuracy_score(y_train, y_pred))
dt_test_acc = accuracy_score(y_test, y_testpred)
print(dt_test_acc)
confusion_matrix(y_train, y_pred)


1.0
0.842


array([[375,   0,   0,   0],
       [  0, 375,   0,   0],
       [  0,   0, 375,   0],
       [  0,   0,   0, 375]])

In [6]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
y_testpred = model.predict(X_test)
print(accuracy_score(y_train, y_pred))
knn_test_acc = accuracy_score(y_test, y_testpred)
print(knn_test_acc)
confusion_matrix(y_train, y_pred)


0.94
0.942


array([[360,  15,   0,   0],
       [ 12, 351,  12,   0],
       [  0,  15, 343,  17],
       [  0,   0,  19, 356]])

In [7]:
model = DummyClassifier(strategy="most_frequent")
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
y_testpred = model.predict(X_test)
print(accuracy_score(y_train, y_pred))
print(accuracy_score(y_test, y_testpred))

0.25
0.25


In [8]:
confusion_matrix(y_train, y_pred)

array([[375,   0,   0,   0],
       [375,   0,   0,   0],
       [375,   0,   0,   0],
       [375,   0,   0,   0]])

2. Confrontare l’accuratezza ottenuta nel punto precedente con l’accuratezza che si ottiene con una 10 Fold cross validation. (punti 1)

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

kf = KFold(n_splits=10, shuffle=True, random_state=42)
model = DecisionTreeClassifier(random_state=42)

scores = [] # Lista per salvare i risultati di ogni "giro"

# kf.split restituisce gli INDICI delle righe
for train_index, test_index in kf.split(X):
    # 1. Selezioniamo i dati veri usando gli indici (.iloc è fondamentale)
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # 2. Alleniamo il modello su QUESTO specifico fold
    model.fit(X_train_fold, y_train_fold)

    # 3. Valutiamo il modello su QUESTO specifico test fold
    y_pred_fold = model.predict(X_test_fold)
    acc = accuracy_score(y_test_fold, y_pred_fold)

    scores.append(acc) # Salviamo il risultato

print("Accuracy per ogni fold:", scores)
print("Media Accuracy:", np.mean(scores))

Accuracy per ogni fold: [0.835, 0.785, 0.805, 0.845, 0.84, 0.8, 0.835, 0.82, 0.87, 0.86]
Media Accuracy: 0.8295


3. Trovare i parametri migliori del classificatore DecisionTree. Agire sui parametri criterion e min_samples_leaf. Verificare se l’accuratezza che si ottiene con la nuova configurazione supera quella ottenuta con i parametri di default al punto 1. (punti 3)

In [10]:
from sklearn.model_selection import GridSearchCV

# 1. Definizione della griglia dei parametri
# Proviamo sia 'gini' che 'entropy'
# Proviamo diversi valori per le foglie (1=default, numeri più alti riducono overfitting)
param_grid = {
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [1, 5, 10, 20, 50, 100]
}

# 2. Configurazione della Grid Search
# cv=5: usa la cross validation a 5 fold per valutare ogni combinazione
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

# 3. Esecuzione della ricerca (Training)
grid_search.fit(X_train, y_train)

# 4. Recupero del miglior modello e dei risultati
best_dt = grid_search.best_estimator_
best_params = grid_search.best_params_
tuned_test_acc = accuracy_score(y_test, best_dt.predict(X_test))

print(f"--- RISULTATI TUNING ---")
print(f"Migliori Parametri trovati: {best_params}")
print(f"Accuracy (Tuned) sul Test:  {tuned_test_acc:.4f}")

--- RISULTATI TUNING ---
Migliori Parametri trovati: {'criterion': 'entropy', 'min_samples_leaf': 5}
Accuracy (Tuned) sul Test:  0.8880


4. Studiare la correlazione tra tutte le feature del dataset, creare un dataframe che contiene, oltre alla colonna target, le 5 feature più correlate (positivamente) al target. Ripetere la classificazione sul nuovo dataset e verificare se l’accuratezza ottenuta con DecisionTree e KNeighborsClassifier migliora (punti 3).

In [11]:
correlation_matrix = df.corr()
top_5_features = correlation_matrix['price_range'].sort_values(ascending=False).index[1:6]
print(top_5_features)
topX = df[top_5_features]
topy = df['price_range']

# Split (stesso random_state per confronto equo)
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(topX, topy, test_size=0.25, random_state=42)

# --- DECISION TREE (Nuovo) ---
dt_new = DecisionTreeClassifier(random_state=42)
dt_new.fit(X_train_new, y_train_new)
acc_dt_new = accuracy_score(y_test_new, dt_new.predict(X_test_new))

# --- KNN (Nuovo) ---
knn_new = KNeighborsClassifier(n_neighbors=5)
knn_new.fit(X_train_new, y_train_new)
acc_knn_new = accuracy_score(y_test_new, knn_new.predict(X_test_new))

# --- STAMPA CONFRONTO ---
print(f"--- CONFRONTO FEATURE SELECTION ---")
print(f"Decision Tree (Tutte le feature): {dt_test_acc:.4f}") # Variabile dall'esercizio 1
print(f"Decision Tree (Top 5 feature):    {acc_dt_new:.4f}")

print(f"\nKNN (Tutte le feature): {knn_test_acc:.4f}")         # Variabile dall'esercizio 1
print(f"KNN (Top 5 feature):    {acc_knn_new:.4f}")


Index(['ram', 'battery_power', 'px_width', 'px_height', 'int_memory'], dtype='object')
--- CONFRONTO FEATURE SELECTION ---
Decision Tree (Tutte le feature): 0.8420
Decision Tree (Top 5 feature):    0.8580

KNN (Tutte le feature): 0.9420
KNN (Top 5 feature):    0.9320


5. Creare una pipeline in cui gli attributi int_memory, ram e talk_time sono scalati in modo che abbiano media 0 e varianza 1, gli attributi mobile_wt e battery_power sono discretizzati in 5 intervalli, e tutti gli altri attributi sono lasciati invariati. La pipeline deve applicare il modello DecisionTree con i parametri migliori trovati al punto 2. Valutare l’accuratezza della classificazione. (punti 3)


In [12]:
# 1. Definisci il modello
estimator = DecisionTreeClassifier(random_state=42, criterion='entropy', min_samples_leaf=5)

# 2. Definisci il ColumnTransformer (Qui decidi QUALI colonne toccare)
coltran = ColumnTransformer(
    transformers=[
        # (nome, trasformatore, lista_colonne)
        ('scaler', StandardScaler(), ["int_memory", "ram", "talk_time"]),
        ('discr', KBinsDiscretizer(n_bins=5, encode='ordinal'), ['mobile_wt', 'battery_power'])
    ],
    remainder='passthrough'  # IMPORTANTE: mantieni le altre colonne non elencate sopra!
)

# 3. Definisci la Pipeline (Prima trasforma le colonne, poi usa il modello)
mypipeline = Pipeline(
    steps=[
        ('preprocessor', coltran),
        ('estimator', estimator)
    ]
)

# 4. Addestra (Passando anche y_train!)
mypipeline.fit(X_train, y_train)

# 5. Valuta
Y_pred = mypipeline.predict(X_test)
print("Accuracy: " + str(accuracy_score(y_test, Y_pred)))
confusion_matrix(y_test, Y_pred)

Accuracy: 0.89


/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


array([[116,   9,   0,   0],
       [  3, 109,  13,   0],
       [  0,   8, 108,   9],
       [  0,   0,  13, 112]])

In [19]:
mypipeline

,steps,"[('preprocessor', ...), ('estimator', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaler', ...), ('discr', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


******6. Aggiungere alla pipeline del punto 5 la funzione SelectKBest (https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest). Utilizzare la funzione di gridSearchCV per selezionare il K migliore e anche il numero migliore di bin in cui discretizzare i valori di mobile_wt e battery_power (scegliere a piacere alcuni valori) (punti 3).******

In [14]:
parameters = {
    'coltran__discr__n_bins': [20, 30, 50],
    'selectkbest__k': [5, "all"],
    'estimator__criterion': ['gini', 'entropy'],
    'estimator__min_samples_leaf': [4, 5, 6, 9, 10]
}

extended_pipeline = Pipeline(steps=[("coltran", coltran),
                                    ("selectkbest", SelectKBest()),
                                    ("estimator", DecisionTreeClassifier(random_state=123))
                                   ])

gd = GridSearchCV(extended_pipeline, parameters)
gd.fit(X_train, y_train)
Y_pred = gd.predict(X_test)
gd.best_params_

/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/Users/gabrieledavena/Uni/Big Data/BData/lib/python3.13/site-packages/sklearn/preprocessing/_discretization.py:296: 

{'coltran__discr__n_bins': 30,
 'estimator__criterion': 'entropy',
 'estimator__min_samples_leaf': 4,
 'selectkbest__k': 5}

In [16]:
gd.best_score_

np.float64(0.8640000000000001)

In [18]:
extended_pipeline

,steps,"[('coltran', ...), ('selectkbest', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scaler', ...), ('discr', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
